# Deep Neural Decision Tree

### Imports

In [9]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

### DNDT Model

In [10]:
def build_neural_network(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu')
    ])
    return model

def train_neural_decision_tree(X_train, y_train, X_test, y_test):
    # Neural network part
    nn_model = build_neural_network((X_train.shape[1],))
    nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    nn_model.fit(X_train, y_train, epochs=10)

    # Get intermediate output from neural network
    intermediate_layer_model = tf.keras.models.Model(inputs=nn_model.input,
                                                     outputs=nn_model.layers[-2].output)
    intermediate_output_train = intermediate_layer_model.predict(X_train)
    intermediate_output_test = intermediate_layer_model.predict(X_test)

    # Decision tree part
    tree_model = DecisionTreeClassifier()
    tree_model.fit(intermediate_output_train, y_train)
    
    # Prediction and evaluation
    y_pred = tree_model.predict(intermediate_output_test)
    return accuracy_score(y_test, y_pred)

### Iris

In [11]:
# Load the Iris dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
column_names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
iris_data = pd.read_csv(url, names=column_names)

# Preprocess the data
X = iris_data.drop("class", axis=1)
y = iris_data["class"]
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2)

# Train the model and evaluate its accuracy
accuracy = train_neural_decision_tree(X_train, y_train, X_test, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/10
4/4 [==============================] - 1s 4ms/step - loss: 9.2602 - accuracy: 0.0500 
Epoch 2/10
4/4 [==============================] - 0s 2ms/step - loss: 6.4649 - accuracy: 0.5167
Epoch 3/10
4/4 [==============================] - 0s 4ms/step - loss: 6.0393 - accuracy: 0.6417
Epoch 4/10
4/4 [==============================] - 0s 4ms/step - loss: 5.7439 - accuracy: 0.6750
Epoch 5/10
4/4 [==============================] - 0s 0s/step - loss: 5.4660 - accuracy: 0.6833
Epoch 6/10
4/4 [==============================] - 0s 172us/step - loss: 5.2032 - accuracy: 0.6833
Epoch 7/10
4/4 [==============================] - 0s 2ms/step - loss: 5.0170 - accuracy: 0.6833
Epoch 8/10
4/4 [==============================] - 0s 2ms/step - loss: 4.9066 - accuracy: 0.6833
Epoch 9/10
4/4 [==============================] - 0s 3ms/step - loss: 4.8320 - accuracy: 0.6833
Epoch 10/10
1/1 [==============================] - 0s 25ms/step
Accuracy: 1.0


### Haberman's Survival

In [13]:
# Load the Haberman's Survival dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/haberman/haberman.data"
column_names = ["age", "year_of_operation", "positive_axillary_nodes", "survival_status"]
haberman_data = pd.read_csv(url, names=column_names)

# Preprocess the data
X = haberman_data.drop("survival_status", axis=1)
y = haberman_data["survival_status"]
# Adjust labels to start from 0
y = y - 1

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

# Train the model and evaluate its accuracy
accuracy = train_neural_decision_tree(X_train, y_train, X_test, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/10
8/8 [==============================] - 1s 2ms/step - loss: 7.0753 - accuracy: 0.1762 
Epoch 2/10
8/8 [==============================] - 0s 2ms/step - loss: 4.6580 - accuracy: 0.7008
Epoch 3/10
8/8 [==============================] - 0s 3ms/step - loss: 2.8218 - accuracy: 0.7336
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 0.9661 - accuracy: 0.7500
Epoch 5/10
8/8 [==============================] - 0s 3ms/step - loss: 0.6294 - accuracy: 0.7541
Epoch 6/10
8/8 [==============================] - 0s 1ms/step - loss: 0.5928 - accuracy: 0.7582
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 0.5698 - accuracy: 0.7582
Epoch 8/10
8/8 [==============================] - 0s 1ms/step - loss: 0.5525 - accuracy: 0.7623
Epoch 9/10
8/8 [==============================] - 0s 1ms/step - loss: 0.5378 - accuracy: 0.7582
Epoch 10/10
2/2 [==============================] - 0s 2ms/step
Accuracy: 0.7258064516129032


### Car Evaluation

In [5]:
# Load the Car Evaluation dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
column_names = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]
car_data = pd.read_csv(url, names=column_names)

# Preprocess the data: One-hot encode categorical features
X = car_data.drop("class", axis=1)
y = car_data["class"]

one_hot_encoder = OneHotEncoder()
X_encoded = one_hot_encoder.fit_transform(X).toarray()

# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2)

# Train the model and evaluate its accuracy
accuracy = train_neural_decision_tree(X_train, y_train, X_test, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/10
44/44 [==============================] - 1s 2ms/step - loss: 2.3928 - accuracy: 0.6006
Epoch 2/10
44/44 [==============================] - 0s 1ms/step - loss: 1.0293 - accuracy: 0.7229
Epoch 3/10
44/44 [==============================] - 0s 862us/step - loss: 0.8732 - accuracy: 0.8473
Epoch 4/10
44/44 [==============================] - 0s 1ms/step - loss: 1.9445 - accuracy: 0.6563
Epoch 5/10
44/44 [==============================] - 0s 1ms/step - loss: 2.0591 - accuracy: 0.6404
Epoch 6/10
44/44 [==============================] - 0s 1ms/step - loss: 2.4263 - accuracy: 0.5260
Epoch 7/10
44/44 [==============================] - 0s 1ms/step - loss: 2.3097 - accuracy: 0.5528
Epoch 8/10
44/44 [==============================] - 0s 1ms/step - loss: 2.0464 - accuracy: 0.6288
Epoch 9/10
44/44 [==============================] - 0s 1ms/step - loss: 2.2083 - accuracy: 0.5825
Epoch 10/10
11/11 [==============================] - 0s 855us/step
Accuracy: 0.8497109826589595


### Breast Cancer Wisconsin (Original)

In [16]:
# Load the Breast Cancer Wisconsin (Original) dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
column_names = ["Sample code number", "Clump Thickness", "Uniformity of Cell Size", "Uniformity of Cell Shape",
                "Marginal Adhesion", "Single Epithelial Cell Size", "Bare Nuclei", "Bland Chromatin",
                "Normal Nucleoli", "Mitoses", "Class"]
cancer_data = pd.read_csv(url, names=column_names)

# Preprocess the data
# Drop the 'Sample code number' as it's not a feature
cancer_data.drop(["Sample code number"], axis=1, inplace=True)

# Replace missing values denoted by '?' with NaN and then impute
cancer_data.replace("?", np.nan, inplace=True)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
cancer_data = pd.DataFrame(imputer.fit_transform(cancer_data), columns=cancer_data.columns)

X = cancer_data.drop("Class", axis=1)
y = cancer_data["Class"]

# Encode the labels (2 for benign, 4 for malignant) to 0 and 1
y_encoded = y.replace({2: 0, 4: 1})

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2)

# Train the model and evaluate its accuracy
accuracy = train_neural_decision_tree(X_train, y_train, X_test, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/10
18/18 [==============================] - 0s 754us/step - loss: 8.0064 - accuracy: 0.4258
Epoch 2/10
18/18 [==============================] - 0s 722us/step - loss: 3.1516 - accuracy: 0.9123
Epoch 3/10
18/18 [==============================] - 0s 727us/step - loss: 2.7506 - accuracy: 0.9535
Epoch 4/10
18/18 [==============================] - 0s 716us/step - loss: 2.6940 - accuracy: 0.9589
Epoch 5/10
18/18 [==============================] - 0s 693us/step - loss: 2.5981 - accuracy: 0.9678
Epoch 6/10
18/18 [==============================] - 0s 779us/step - loss: 2.5919 - accuracy: 0.9678
Epoch 7/10
18/18 [==============================] - 0s 770us/step - loss: 2.5597 - accuracy: 0.9678
Epoch 8/10
18/18 [==============================] - 0s 749us/step - loss: 2.5508 - accuracy: 0.9553
Epoch 9/10
18/18 [==============================] - 0s 795us/step - loss: 2.4923 - accuracy: 0.9589
Epoch 10/10
5/5 [==============================] - 0s 554us/step
Accuracy: 0.9285714285714286


### Pima Indians Diabetes

In [19]:
# Load the Pima Indians Diabetes dataset
diabetes_data = pd.read_csv('./datasets/diabetes.csv')

# Preprocess the data
# Replace zeros in certain columns with NaN and then impute
columns_to_impute = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
diabetes_data[columns_to_impute] = diabetes_data[columns_to_impute].replace(0, np.nan)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
diabetes_data[columns_to_impute] = imputer.fit_transform(diabetes_data[columns_to_impute])

X = diabetes_data.drop("Outcome", axis=1)
y = diabetes_data["Outcome"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

# Train the model and evaluate its accuracy
accuracy = train_neural_decision_tree(X_train, y_train, X_test, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/10
20/20 [==============================] - 1s 2ms/step - loss: 2.7498 - accuracy: 0.4251 
Epoch 2/10
20/20 [==============================] - 0s 2ms/step - loss: 0.7299 - accuracy: 0.7459
Epoch 3/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5276 - accuracy: 0.7573
Epoch 4/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5364 - accuracy: 0.7687
Epoch 5/10
20/20 [==============================] - 0s 2ms/step - loss: 0.4959 - accuracy: 0.7704
Epoch 6/10
20/20 [==============================] - 0s 1ms/step - loss: 0.4813 - accuracy: 0.7671
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5336 - accuracy: 0.7850
Epoch 8/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5024 - accuracy: 0.7818
Epoch 9/10
20/20 [==============================] - 0s 2ms/step - loss: 0.4964 - accuracy: 0.7850
Epoch 10/10
5/5 [==============================] - 0s 3ms/step
Accuracy: 0.7142857142857143


### Poker Hands

In [14]:
# Load the Poker Hand dataset
url_train = "https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-training-true.data"
url_test = "https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-testing.data"
column_names = ["Suit1", "Card1", "Suit2", "Card2", "Suit3", "Card3", "Suit4", "Card4", "Suit5", "Card5", "Class"]
poker_data_train = pd.read_csv(url_train, names=column_names)
poker_data_test = pd.read_csv(url_test, names=column_names)

# Preprocess the data: One-hot encode categorical features
X_train = poker_data_train.drop("Class", axis=1)
y_train = poker_data_train["Class"]
X_test = poker_data_test.drop("Class", axis=1)
y_test = poker_data_test["Class"]

one_hot_encoder = OneHotEncoder()
X_train_encoded = one_hot_encoder.fit_transform(X_train).toarray()
X_test_encoded = one_hot_encoder.transform(X_test).toarray()

# Train the model and evaluate its accuracy
accuracy = train_neural_decision_tree(X_train_encoded, y_train, X_test_encoded, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/10
782/782 [==============================] - 1s 740us/step - loss: 1.9310 - accuracy: 0.4875
Epoch 2/10
782/782 [==============================] - 1s 801us/step - loss: 1.8242 - accuracy: 0.5154
Epoch 3/10
782/782 [==============================] - 1s 712us/step - loss: 2.5727 - accuracy: 0.5285
Epoch 4/10
782/782 [==============================] - 1s 708us/step - loss: 3.4655 - accuracy: 0.4995
Epoch 5/10
782/782 [==============================] - 1s 703us/step - loss: 3.4655 - accuracy: 0.4995
Epoch 6/10
782/782 [==============================] - 1s 698us/step - loss: 3.4655 - accuracy: 0.4995
Epoch 7/10
782/782 [==============================] - 1s 698us/step - loss: 3.4655 - accuracy: 0.4995
Epoch 8/10
782/782 [==============================] - 1s 710us/step - loss: 3.4655 - accuracy: 0.4995
Epoch 9/10
782/782 [==============================] - 1s 702us/step - loss: 3.4655 - accuracy: 0.4995
Epoch 10/10
31250/31250 [==============================] - 10s 320us/step
Accuracy

### Statlog (German Credit)

In [58]:
# Load the German Credit dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
column_names = ["Status of existing checking account", "Duration in month", "Credit history", 
                "Purpose", "Credit amount", "Savings account/bonds", "Present employment since", 
                "Installment rate in percentage of disposable income", "Personal status and sex", 
                "Other debtors / guarantors", "Present residence since", "Property", 
                "Age in years", "Other installment plans", "Housing", "Number of existing credits at this bank", 
                "Job", "Number of people being liable to provide maintenance for", "Telephone", 
                "Foreign worker", "Credit risk"]
credit_data = pd.read_csv(url, names=column_names, sep=' ', header=None)

# Identify categorical and numerical columns
categorical_cols = ["Status of existing checking account", "Credit history", "Purpose", "Savings account/bonds", 
                    "Present employment since", "Personal status and sex", "Other debtors / guarantors", 
                    "Property", "Other installment plans", "Housing", "Job", "Telephone", "Foreign worker"]
numerical_cols = ["Duration in month", "Credit amount", "Installment rate in percentage of disposable income", 
                  "Present residence since", "Age in years", "Number of existing credits at this bank", 
                  "Number of people being liable to provide maintenance for"]

# Preprocess the data
X = credit_data.drop("Credit risk", axis=1)
y = credit_data["Credit risk"]

# Create a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

X_processed = preprocessor.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2)

# Train the model and evaluate its accuracy
accuracy = train_neural_decision_tree(X_train, y_train, X_test, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/10
25/25 [==============================] - 0s 710us/step - loss: 2.5563 - accuracy: 0.5263
Epoch 2/10
25/25 [==============================] - 0s 678us/step - loss: 0.7060 - accuracy: 0.7075
Epoch 3/10
25/25 [==============================] - 0s 675us/step - loss: 0.5837 - accuracy: 0.7063
Epoch 4/10
25/25 [==============================] - 0s 704us/step - loss: 0.5151 - accuracy: 0.7088
Epoch 5/10
25/25 [==============================] - 0s 764us/step - loss: 0.4975 - accuracy: 0.7063
Epoch 6/10
25/25 [==============================] - 0s 759us/step - loss: 0.4701 - accuracy: 0.7138
Epoch 7/10
25/25 [==============================] - 0s 758us/step - loss: 0.4487 - accuracy: 0.7113
Epoch 8/10
25/25 [==============================] - 0s 733us/step - loss: 0.4287 - accuracy: 0.7125
Epoch 9/10
25/25 [==============================] - 0s 774us/step - loss: 0.4152 - accuracy: 0.7125
Epoch 10/10
7/7 [==============================] - 0s 514us/step
Accuracy: 0.755


### Connect-4

In [41]:
# Load the Connect-4 dataset
path_to_dataset = './datasets/connect-4.csv'
column_names = ["C"+str(i) for i in range(1, 43)] + ["Class"]
connect4_data = pd.read_csv(path_to_dataset, names=column_names, header=None)

# Preprocess the data: One-hot encode categorical features
X = connect4_data.drop("Class", axis=1)
y = connect4_data["Class"]

one_hot_encoder = OneHotEncoder()
X_encoded = one_hot_encoder.fit_transform(X).toarray()

# Encode the labels using LabelEncoder from scikit-learn
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Train the model and evaluate its accuracy
accuracy = train_neural_decision_tree(X_train, y_train, X_test, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/10
1689/1689 [==============================] - 3s 1ms/step - loss: 1.0714 - accuracy: 0.2414
Epoch 2/10
1689/1689 [==============================] - 2s 1ms/step - loss: 1.0986 - accuracy: 0.2461
Epoch 3/10
1689/1689 [==============================] - 2s 1ms/step - loss: 1.0986 - accuracy: 0.2461
Epoch 4/10
1689/1689 [==============================] - 2s 1ms/step - loss: 1.0986 - accuracy: 0.2461
Epoch 5/10
1689/1689 [==============================] - 2s 1ms/step - loss: 1.0986 - accuracy: 0.2461
Epoch 6/10
1689/1689 [==============================] - 2s 1ms/step - loss: 1.0986 - accuracy: 0.2461
Epoch 7/10
1689/1689 [==============================] - 2s 1ms/step - loss: 1.0986 - accuracy: 0.2461
Epoch 8/10
1689/1689 [==============================] - 2s 1ms/step - loss: 1.0986 - accuracy: 0.2461
Epoch 9/10
1689/1689 [==============================] - 2s 1ms/step - loss: 1.0986 - accuracy: 0.2461
Epoch 10/10
423/423 [==============================] - 1s 1ms/step
Accuracy: 0.664

### Image Segmentation

In [33]:
# Load the Image Segmentation dataset
path_to_dataset = './datasets/segmentation_train.csv'
column_names = ["Class", "Region-centroid-col", "Region-centroid-row", "Region-pixel-count", "Short-line-density-5", 
                "Short-line-density-2", "Vedge-mean", "Vegde-sd", "Hedge-mean", "Hedge-sd", "Intensity-mean", 
                "Rawred-mean", "Rawblue-mean", "Rawgreen-mean", "Exred-mean", "Exblue-mean", "Exgreen-mean", 
                "Value-mean", "Saturation-mean", "Hue-mean"]
segmentation_data = pd.read_csv(path_to_dataset, names=column_names, header=None, skiprows=5)

# Preprocess the data
X = segmentation_data.drop("Class", axis=1)
y = segmentation_data["Class"]

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2)

# Train the model and evaluate its accuracy
accuracy = train_neural_decision_tree(X_train, y_train, X_test, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/10
6/6 [==============================] - 1s 4ms/step - loss: 8.3564 - accuracy: 0.2012
Epoch 2/10
6/6 [==============================] - 0s 2ms/step - loss: 6.7408 - accuracy: 0.4085
Epoch 3/10
6/6 [==============================] - 0s 2ms/step - loss: 5.7669 - accuracy: 0.5427
Epoch 4/10
6/6 [==============================] - 0s 457us/step - loss: 4.2955 - accuracy: 0.5976
Epoch 5/10
6/6 [==============================] - 0s 537us/step - loss: 3.7014 - accuracy: 0.6220
Epoch 6/10
6/6 [==============================] - 0s 3ms/step - loss: 2.7491 - accuracy: 0.5915
Epoch 7/10
6/6 [==============================] - 0s 4ms/step - loss: 2.4359 - accuracy: 0.5061
Epoch 8/10
6/6 [==============================] - 0s 2ms/step - loss: 2.1847 - accuracy: 0.4939
Epoch 9/10
6/6 [==============================] - 0s 0s/step - loss: 2.0940 - accuracy: 0.5366
Epoch 10/10
2/2 [==============================] - 0s 3ms/step
Accuracy: 0.7317073170731707


### Covrtype

In [38]:
# Load the Covertype dataset
path_to_dataset = './datasets/covtype.csv'
column_names = [f'feature_{i}' for i in range(1, 55)]
covertype_data = pd.read_csv(path_to_dataset, names=column_names)

# Preprocess the data
# Assuming the last column is the target variable
X = covertype_data.iloc[:, :-1]
y = covertype_data.iloc[:, -1]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train the model and evaluate its accuracy
accuracy = train_neural_decision_tree(X_train, y_train, X_test, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/10
14526/14526 [==============================] - 18s 1ms/step - loss: 2.8946 - accuracy: 0.1493
Epoch 2/10
14526/14526 [==============================] - 17s 1ms/step - loss: 3.4091 - accuracy: 0.0157
Epoch 3/10
14526/14526 [==============================] - 17s 1ms/step - loss: 3.4092 - accuracy: 0.0157
Epoch 4/10
14526/14526 [==============================] - 17s 1ms/step - loss: 3.4092 - accuracy: 0.0157
Epoch 5/10
14526/14526 [==============================] - 17s 1ms/step - loss: 3.4092 - accuracy: 0.0157
Epoch 6/10
14526/14526 [==============================] - 16s 1ms/step - loss: 3.4092 - accuracy: 0.0157
Epoch 7/10
14526/14526 [==============================] - 17s 1ms/step - loss: 3.4092 - accuracy: 0.0157
Epoch 8/10
14526/14526 [==============================] - 17s 1ms/step - loss: 3.4092 - accuracy: 0.0157
Epoch 9/10
14526/14526 [==============================] - 17s 1ms/step - loss: 3.4092 - accuracy: 0.0157
Epoch 10/10
3632/3632 [==============================] 